# **MapReduce** et **WordCount**

Notebook juste pour refaire les exemple du cours sur Spark.

In [1]:
from collections import defaultdict

def wordCount(text):
    counts = defaultdict(int)
    for word in text.split():
        counts[word.lower()] += 1
    return counts

In [2]:
D1 = {"./lot1.txt": "jour lève notre grisaille"}
D2 = {"./lot2.txt": "trottoir notre ruelle notre tour"}
D3 = {"./lot3.txt": "jour lève notre envie vous"}
D4 = {"./lot4.txt": "faire comprendre tous notre tour"}

In [7]:
def _map(key, value):
    return key, [(word, 1) for word in value.split()]

In [8]:
def _reduce(key, values):
    result = sum(values)
    return key, result

In [5]:
corpus = {
    "lot1": "jour lève notre grisaille",
    "lot2": "trottoir notre ruelle notre tour",
    "lot3": "jour lève notre envie vous",
    "lot4": "faire comprendre tous notre tour"
}

In [13]:
mapped = {
    key: _map(key, value)[1]
    for key, value in corpus.items()
}
display(mapped)

{'lot1': [('jour', 1), ('lève', 1), ('notre', 1), ('grisaille', 1)],
 'lot2': [('trottoir', 1),
  ('notre', 1),
  ('ruelle', 1),
  ('notre', 1),
  ('tour', 1)],
 'lot3': [('jour', 1), ('lève', 1), ('notre', 1), ('envie', 1), ('vous', 1)],
 'lot4': [('faire', 1),
  ('comprendre', 1),
  ('tous', 1),
  ('notre', 1),
  ('tour', 1)]}

In [18]:
def _shuffle_sort(mapped):
    shuffled = {}
    for pair_list in mapped.values():
        for k, v in pair_list:
            if k in shuffled:
                shuffled[k] += v
            else:
                shuffled[k] = v
    return shuffled
_shuffle_sort(mapped)

{'jour': 2,
 'lève': 2,
 'notre': 5,
 'grisaille': 1,
 'trottoir': 1,
 'ruelle': 1,
 'tour': 2,
 'envie': 1,
 'vous': 1,
 'faire': 1,
 'comprendre': 1,
 'tous': 1}

In [1]:
from pyspark import SparkContext

# Instantiation d'un SparkContext
sc = SparkContext()

# Lecture d'un fichier texte : le fichier est décomposé en lignes.
lines = sc.textFile("Discours de la méthode.txt") 
display(lines.take(10))

# Décomposition de chaque ligne en mots (et aplatissement)
tokenized_lines = lines.flatMap(lambda line: line.split(' '))
display(tokenized_lines.take(10))

# Chacun des mots est transformé en une clé-valeur
mapped_lines = tokenized_lines.map(lambda word: (word, 1))
display(mapped_lines.take(10))

# Les valeurs associées à chaque clé sont sommées
word_counts = mapped_lines.reduceByKey(lambda count1, count2: count1 + count2)

# This word_counts is a RDD
display(word_counts)
#word_counts.foreachPartition(lambda p: print(list(p)[:10]))

# Le résultat est récupéré => lazy eval
word_counts = word_counts.collect()

# Chaque paire (clé, valeur) est affichée
for (word, count) in word_counts[:10]:
    print(word, count)

['DISCOURS DE LA METHODE',
 '',
 'POUR BIEN CONDUIRE SA RAISON,',
 '',
 'ET CHERCHER LA VERITE DANS LES SCIENCES',
 '',
 '',
 "Si ce discours semble trop long pour �tre lu en une fois, on le pourra distinguer en six parties. Et, en la premi�re, on trouvera diverses consid�rations touchant les sciences. En la seconde, les principales r�gles de la m�thode que l'auteur a cherch�e. En la troisi�me, quelques unes de celles de la morale qu'il a tir�e de cette m�thode. En la quatri�me, les raisons par lesquelles il prouve l'existence de Dieu et de l'�me humaine, qui sont les fondements de sa m�taphysique. En la cinqui�me, l'ordre des questions de physique qu'il a cherch�es, et particuli�rement l'explication des mouvements du coeur et de quelques autres difficult�s qui appartiennent � la m�decine; puis aussi la diff�rence qui est entre notre �me et celle des b�tes. Et en la derni�re, quelles choses il croit �tre requises pour aller plus avant en la recherche de la nature qu'il n'a �t�, et quel

['DISCOURS',
 'DE',
 'LA',
 'METHODE',
 '',
 'POUR',
 'BIEN',
 'CONDUIRE',
 'SA',
 'RAISON,']

[('DISCOURS', 1),
 ('DE', 1),
 ('LA', 1),
 ('METHODE', 1),
 ('', 1),
 ('POUR', 1),
 ('BIEN', 1),
 ('CONDUIRE', 1),
 ('SA', 1),
 ('RAISON,', 1)]

PythonRDD[9] at RDD at PythonRDD.scala:53

LA 2
METHODE 1
 92
POUR 1
BIEN 1
CONDUIRE 1
ET 1
VERITE 1
LES 1
ce 159
